In [1]:
from icrl import *

## Setup 1

In [2]:
num_actions = 10
conext_dim = 5
env = Environment(num_actions, conext_dim)

# LinUCB
linucb = LinUCB(num_actions, conext_dim)


100,000 trajactories, 200 time steps each

In [3]:
import tqdm
T = 200
num_trajectories = 100000

trajectories = [] # list of list of tuples [[(state,action,reward),...],...]
# In this setting, s_t = \mathbb{A} = action_set!
all_regrets = np.zeros((num_trajectories, T))

# use tqdm to show progress bar

for i in tqdm.tqdm(range(num_trajectories)):
    trajectory = []
    total_regret = 0
    regrets = []
    best_action_index = env.get_best_action_index()  # Best action doesn't change in this setup
    best_action_reward = np.dot(env.action_set[best_action_index], env.w_star)
    for _ in range(T):
        action_index = linucb.select_action(env.action_set)
        reward, action = env.step(action_index)
        # find action
        linucb.update(action_index, reward, action)
        
        # Calculate regret for this round and add to total
        round_regret = best_action_reward - reward
        total_regret += round_regret
        
        trajectory.append((env.get_action_set(), action, reward))
        regrets.append(total_regret)

    all_regrets[i] = regrets # Store regrets for this trajectory
    trajectories.append(trajectory) # Store trajectory

  8%|▊         | 8344/100000 [17:10<3:15:52,  7.80it/s]

In [ ]:
# Use dataframes to store the data
import pandas as pd
df_regrets= pd.DataFrame(all_regrets)
df_regrets.to_csv('linucb_regrets.csv', index=False)


# Save trajectories
import pickle
with open('linucb_trajectories.pkl', 'wb') as f:
    pickle.dump(trajectories, f)

In [ ]:
# Calculate average regret and standard deviation
average_regrets = np.mean(all_regrets, axis=0)
std_regrets = np.std(all_regrets, axis=0)

# 